In [1]:
import re
import math
import warnings
import tqdm
import json
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import optimize


warnings.filterwarnings("ignore")

/root/miniconda3/envs/jupyter-env/lib/python3.11/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
data_=pd.read_excel('1a数据_已替换时间戳_已矫正.xlsx')
data_

,入院首次影像检查流水号,HM_volume,随访1流水号,HM_volume.1,随访2流水号,HM_volume.2,随访3流水号,HM_volume.3,随访4流水号,HM_volume.4,随访5流水号,HM_volume.5,随访6流水号,HM_volume.6,随访7流水号,HM_volume.7,随访8流水号,HM_volume.8
0,9000,69714,29766.0,74902.0,475590.0,70952.0,935055.0,62831.0,1531938.0,44029.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10800,47500,53724.0,52271.0,249199.0,47748.0,1612859.0,13055.0,4029091.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7200,86396,34281.0,106042.0,142545.0,103263.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3600,45498,61107.0,39877.0,301843.0,16622.0,819004.0,8441.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18000,14832,95283.0,24472.0,352595.0,25477.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,19800,32559,89575.0,35138.0,452217.0,34241.0,1055942.0,14377.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,1800,18150,4920.0,37650.0,25620.0,140688.0,72240.0,123926.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,10080,27969,66180.0,27071.0,256680.0,24119.0,1111140.0,3647.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158,10800,53154,95520.0,126642.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
data1 = pd.read_excel('2c治疗方案.xlsx') # 治疗方案
data2 = pd.read_excel('2a数据_已替换时间戳_已矫正.xlsx') # 水肿数据

In [126]:
data3 = pd.merge(data1, data2, how='outer', on='ID')
data3 = data3.drop(range(101, 131))

In [127]:
data3.columns

Index(['ID', '脑室引流', '止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经',
       '发病到首次影像检查时间间隔', '入院首次影像检查流水号', 'ED_volume.0', '随访1流水号', 'ED_volume.1',
       '随访2流水号', 'ED_volume.2', '随访3流水号', 'ED_volume.3', '随访4流水号',
       'ED_volume.4', '随访5流水号', 'ED_volume.5', '随访6流水号', 'ED_volume.6',
       '随访7流水号', 'ED_volume.7', '随访8流水号', 'ED_volume.8'],
      dtype='object')

In [128]:
import json
with open('best_labels_130.json','r') as f:
    groups_label=json.load(f)
print(len(groups_label['best_labels_130']))

130


In [129]:
dataT=data3
dataT['分组信息']=groups_label['best_labels_130']
dataT.to_excel('含分组信息数据.xlsx',index=False)

In [130]:
# 创建一个新列"差"，初始化为0
dataT['差'] = 0
U=globals()
res={}
for i in list(set(dataT['分组信息'])):
    U[f'data_g{i}']=dataT[dataT['分组信息']==i]

# 遍历数据集的每一行
    for index, row in U[f'data_g{i}'].iterrows():
        row_sum = 0  # 用于累积每行的差值

        prev_value = None  # 用于存储前一个以"ED"开头的列的值

        # 遍历以"ED"开头的列
        for col in row.index:
            if col.startswith('ED'):
                value = row[col]
                if not pd.isna(value):  # 确保元素不是NaN
                    if prev_value is not None:
                        row_sum += (value - prev_value)  # 计算差值并累积
                    prev_value = value

        # 根据row_sum的值设置"差"列的值
        if row_sum > 0:
            U[f'data_g{i}'].at[index, '差'] = 1
        elif row_sum < 0:
            U[f'data_g{i}'].at[index, '差'] = -1
        res[f'组{i}']=float(U[f'data_g{i}']['差'].sum())

res_res = {}
for i in res:
    if res[i] < 0:
        res_res[i]=-1
    else:
        res_res[i]=1
# with open('分组趋势.json', 'w') as f:
#     json.dump(res_res,f)

In [131]:
# with open('分组趋势.json', 'r') as f:
#     print(json.load(f))

In [132]:
res_res

{'组0': 1, '组1': 1, '组2': -1, '组3': -1, '组4': -1}

In [133]:
#################### 合并最终表

In [134]:
data4 = pd.read_excel("治疗方案.xlsx")
data4 = data4.drop(range(101, 131))
data4.head()

,ID,"['止吐护胃', '营养神经']","['降颅压治疗', '降压治疗']","['止血治疗', '止吐护胃']","['止血治疗', '降颅压治疗', '降压治疗']","['止血治疗', '降压治疗', '营养神经']","['止血治疗', '镇静、镇痛治疗', '止吐护胃']","['降颅压治疗', '降压治疗', '止吐护胃']","['镇静、镇痛治疗', '止吐护胃', '营养神经']","['降压治疗', '止吐护胃', '营养神经']",...,"['止血治疗', '降压治疗', '止吐护胃', '营养神经']","['止血治疗', '降颅压治疗', '降压治疗', '止吐护胃', '营养神经']","['止血治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃']","['降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '降颅压治疗', '降压治疗', '止吐护胃', '营养神经']","['止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']"
0,sub001,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,0,1,0,0
1,sub002,1,1,1,1,1,0,1,0,1,...,1,1,0,0,0,0,0,0,0,0
2,sub003,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,0,1,0,0
3,sub004,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,sub005,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [135]:
target = []
for count, key in enumerate(groups_label['best_labels_130']):
    value = res_res['组'+str(key)]
    target.append(value)

In [136]:
data4['组趋势'] = 0
data4['组趋势'] = np.array(target)
data4 = data4.drop(['ID'], axis=1)
data4.head()

,"['止吐护胃', '营养神经']","['降颅压治疗', '降压治疗']","['止血治疗', '止吐护胃']","['止血治疗', '降颅压治疗', '降压治疗']","['止血治疗', '降压治疗', '营养神经']","['止血治疗', '镇静、镇痛治疗', '止吐护胃']","['降颅压治疗', '降压治疗', '止吐护胃']","['镇静、镇痛治疗', '止吐护胃', '营养神经']","['降压治疗', '止吐护胃', '营养神经']","['止血治疗', '止吐护胃', '营养神经']",...,"['止血治疗', '降颅压治疗', '降压治疗', '止吐护胃', '营养神经']","['止血治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃']","['降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '降颅压治疗', '降压治疗', '止吐护胃', '营养神经']","['止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']",组趋势
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,1,0,0,1
1,1,1,1,1,1,0,1,0,1,1,...,1,0,0,0,0,0,0,0,0,-1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,1,0,0,1
3,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
4,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,-1


In [137]:
import statsmodels.api as sm


# 选择自变量（X）和因变量（Y）
X = data4.drop('组趋势', axis=1)
Y = data4['组趋势']

# 2. 添加截距（常数项）
X = sm.add_constant(X)

# 3. 拟合多元线性回归模型
model = sm.OLS(Y, X).fit()

# 4. 获取回归结果摘要
summary = model.summary()

# 5. 打印结果摘要
print(summary)
# 指定保存文件的路径和名称
file_path = "regression_summary.txt"
# 打开文件以写入模式
with open(file_path, "w") as file:
    # 将回归结果摘要写入文件
    file.write(str(summary))
# 提示保存成功
print(f"回归结果摘要已保存到 {file_path}")

                            OLS Regression Results                            
Dep. Variable:                    组趋势   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 3.712e+28
Date:                Sun, 24 Sep 2023   Prob (F-statistic):               0.00
Time:                        04:09:22   Log-Likelihood:                 4002.4
No. Observations:                 130   AIC:                            -7953.
Df Residuals:                     104   BIC:                            -7878.
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                                                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------